In [24]:
import pandas as pd
from bs4 import BeautifulSoup
import codecs
from urllib.request import urlparse, urljoin
import requests

In [25]:
#f = codecs.open('metacartel_forum.html', 'r')
#rawdata = f.read()

In [26]:
def scrape_forum(soup, domain):

    urls = []
    
    for a_tag in soup.findAll("a"):
        
        href = a_tag.attrs.get("href")
        
        if href == "" or href is None:
            continue
            
        parsed_href = urlparse(href)
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        
        if href in urls:
            continue
            
        if domain not in href:
            continue
            
        if '/t/' not in href:
            continue

        urls.append(href)

    return urls

In [27]:
urls = []
for page in range(1,7):
    
    print('Scraping page ', page)
    
    response = requests.get(f'https://forum.metacartel.org/c/dao/7/l/latest.html?ascending=false&page={page}')
    rawdata = response.text
    
    soup = BeautifulSoup(rawdata, 'html.parser')
    
    urls.extend(scrape_forum(soup, 'metacartel'))

Scraping page  1
Scraping page  2
Scraping page  3
Scraping page  4
Scraping page  5
Scraping page  6


In [28]:
forum_links = []
titles = []
authors = []
descriptions = []
comments = []
post_times = []

for link in urls[2:]:
    
    print(link)

    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")

    title = soup.find_all("title")[0].get_text()

    first_post = soup.find_all('div', {"itemtype": 'http://schema.org/DiscussionForumPosting'})[0]

    author = first_post.find('a').get_text()

    post_time = first_post.find('time')['datetime']

    desc_text = ''

    for p in first_post.find_all("p"):
        desc_text += ' '+p.get_text()


    comments_this_project = []

    for post in soup.find_all('div', {"itemtype": 'http://schema.org/DiscussionForumPosting'})[1:]:
        comment = post.find("p")
        comment_author = post.find('a').get_text()
        comments_this_project.append((comment, comment_author))


    forum_links.append(link)
    titles.append(title)
    authors.append(author)
    descriptions.append(desc_text)
    comments.append(comments_this_project)
    post_times.append(post_time)


https://forum.metacartel.org/t/proposal-saints-and-sinners/1747
https://forum.metacartel.org/t/yapeswap-dao-erc-1155-erc-20-compatible-amm-pools/1727
https://forum.metacartel.org/t/gig-protocol-grant-proposal/1628
https://forum.metacartel.org/t/truckin-coins-proposal/1703
https://forum.metacartel.org/t/tokenize-community-engagement/1658
https://forum.metacartel.org/t/behind-the-handle-funding/1637
https://forum.metacartel.org/t/proposal-decentralized-block-explorer/862
https://forum.metacartel.org/t/proposal-chess-crypto-cultivation/1283
https://forum.metacartel.org/t/proposal-diamond-decentralized-data-driven-media-collective/1236
https://forum.metacartel.org/t/let-the-nfts-fight-to-see-who-is-the-best/1585
https://forum.metacartel.org/t/proposal-animal-creator-token/973
https://forum.metacartel.org/t/proposal-dao-universal-investment-platform/1310
https://forum.metacartel.org/t/proposal-randchain-a-community-driven-decentralised-randomness-beacon/1304
https://forum.metacartel.org/t/l

https://forum.metacartel.org/t/dao-proposal-give-darren-shares/402
https://forum.metacartel.org/t/dao-proposal-solui/408
https://forum.metacartel.org/t/signal-dao-a-first-experiment-in-daoops/405
https://forum.metacartel.org/t/dao-proposal-ethdenver-activities-d/403
https://forum.metacartel.org/t/support-our-spicy-summoner/385
https://forum.metacartel.org/t/dao-proposal-cooper-turley-membership-grant/342
https://forum.metacartel.org/t/metagame-proposal-v0-12/319
https://forum.metacartel.org/t/dao-proposal-wallet-notify-a-standardized-way-of-sending-push-notifications-to-ethereum-wallets/332
https://forum.metacartel.org/t/dao-proposal-partial-funding-for-university-dao-course-content-development/288
https://forum.metacartel.org/t/dao-proposal-the-dogs-molochs-dao-dashboard-for-daosaka/269
https://forum.metacartel.org/t/dao-proposal-membership-proposal-mediate-app-as-metacartel-rev-gen-dao-prototype/285
https://forum.metacartel.org/t/dao-proposal-ethwaterloo-bounty-for-burner-wallet-2-pl

In [29]:
data = pd.DataFrame.from_dict({'forum_link': forum_links, 'title': titles, 'author': authors, 
                               'description': descriptions, 'comments': comments, 'time_posted': post_times})

In [30]:
data.head()

,forum_link,title,author,description,comments,time_posted
0,https://forum.metacartel.org/t/proposal-saints...,Proposal: Saints and Sinners - 🔥 DAO Proposals...,rg203,Title: Saints and Sinners Description: We are...,"[([Super cool! kinda feels like Wargames, but ...",2021-11-12T23:36:19Z
1,https://forum.metacartel.org/t/yapeswap-dao-er...,Yapeswap DAO: ERC-1155/ERC-20 Compatible AMM p...,Torg,Description\nWe are building liquidity pools ...,[],2021-11-10T02:11:25Z
2,https://forum.metacartel.org/t/gig-protocol-gr...,Gig Protocol Grant Proposal - 🔥 DAO Proposals ...,jacobean,"Hi everyone, thank you for the opportunity to...","[([Hey , <a class=""mention"" href=""/u/jacobean""...",2021-10-19T04:42:07Z
3,https://forum.metacartel.org/t/truckin-coins-p...,Truckin' Coins Proposal - 🔥 DAO Proposals - 🤩 ...,TruckinCoins,Truckin’ Coins Problem\nAs you can currently ...,[],2021-11-06T09:48:20Z
4,https://forum.metacartel.org/t/tokenize-commun...,Tokenize Community Engagement - 🔥 DAO Proposal...,Autocrat,\nPath to Oblivion1638×2048 355 KB\n Proo...,"[([Had a good chat with , <a class=""mention"" h...",2021-10-27T22:45:35Z


In [23]:
data.to_csv('scraped_forum_data.csv')